# Sentiment analysis para comentarios de peliculas 
#### Proyecto parte 3
Ruben Gonzalez 20003314

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.metrics import roc_auc_score

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.callbacks import EarlyStopping

from keras.optimizers import Adam

import keras.models
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, BatchNormalization, GlobalMaxPooling1D, Dropout
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-reviews-dataset/imdb_reviews_dataset.csv


In [2]:
import tensorflow as tf

## Carga y preprocesamiento de datos

In [3]:
df = pd.read_csv('../input/imdb-reviews-dataset/imdb_reviews_dataset.csv')
df.head()

,data_type,id,rating,text
0,train,pos_0,9,Bromwell High is a cartoon comedy. It ran at t...
1,train,pos_10000,8,Homelessness (or Houselessness as George Carli...
2,train,pos_10001,10,Brilliant over-acting by Lesley Ann Warren. Be...
3,train,pos_10002,7,This is easily the most underrated film inn th...
4,train,pos_10003,8,This is not the typical Mel Brooks film. It wa...


In [4]:
df.rating.value_counts()

0     50000
1     10122
10     9731
8      5859
4      5331
3      4961
7      4803
9      4607
2      4586
Name: rating, dtype: int64

Se remueven los reviews sin rating pues no nos sirven para el analisis

In [5]:
df_etiquetado = df[df['rating'] != 0].copy() 
df_etiquetado.shape

(50000, 4)

In [6]:
df_etiquetado.rating.value_counts()

1     10122
10     9731
8      5859
4      5331
3      4961
7      4803
9      4607
2      4586
Name: rating, dtype: int64

Se crea una nueva columna que identifique si el sentimiento es positivo o no usando aquellos comentarios con ranquin mayor o igual a 7 como positivos

In [7]:
df_etiquetado['sentimiento'] = df_etiquetado.rating.apply(lambda x: 1 if x >= 7 else 0)
df_etiquetado["Length"] = df_etiquetado['text'].apply(lambda x: len(x.split()))

In [8]:
df_etiquetado.head()

,data_type,id,rating,text,sentimiento,Length
0,train,pos_0,9,Bromwell High is a cartoon comedy. It ran at t...,1,140
1,train,pos_10000,8,Homelessness (or Houselessness as George Carli...,1,428
2,train,pos_10001,10,Brilliant over-acting by Lesley Ann Warren. Be...,1,147
3,train,pos_10002,7,This is easily the most underrated film inn th...,1,124
4,train,pos_10003,8,This is not the typical Mel Brooks film. It wa...,1,120


Como el dataset ya esta segmentado en train y test, se usara esa misma segmentacion

In [9]:
len_text_info = df_etiquetado['Length'].describe()
max_length = int(len_text_info['mean'] + 2 * len_text_info['std'])
print(max_length)

573


In [10]:
df_train = df_etiquetado[df_etiquetado.data_type == "train"]
df_test = df_etiquetado[df_etiquetado.data_type != "train"]

In [11]:
print(len(df_train))
print(len(df_test))

25000
25000


La segmentacion es mala dado que es la mitad train y la mitad test, esto no es recomendado por lo tanto se realizara mejor un split teniendo tambien dataset de validacion

In [17]:
X = df_etiquetado.iloc[:,-3:-2]
X.head()

,text
0,Bromwell High is a cartoon comedy. It ran at t...
1,Homelessness (or Houselessness as George Carli...
2,Brilliant over-acting by Lesley Ann Warren. Be...
3,This is easily the most underrated film inn th...
4,This is not the typical Mel Brooks film. It wa...


In [18]:
Y = df_etiquetado.iloc[:,-2:-1]
Y.head()

,sentimiento
0,1
1,1
2,1
3,1
4,1


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.02, random_state = 0)

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.05, random_state = 0)

In [23]:
X_train.head()

,text
85866,"Well, first of all - i am a big fanatic of hor..."
79278,"The ""Hunting Trilogy"" of Rabbit Fire (1951), R..."
99323,If you feel like wasting 86 minutes on a film ...
22917,I didn't think this was as absolutely horrible...
88243,I truly hate and despise this film and the fil...


## Pre-procesamiento del texto

In [24]:
def split_long_texts(text_tokens, labels):
    _text_tokens = []
    _labels = []
    for text, label in zip(text_tokens, labels):
        text_size = len(text) // max_length
        text_res = len(text) % max_length

        for i in range(text_size):
            _text_tokens.append(text[i: i + max_length])
            _labels.append(label)

        if text_res > 0.5 * max_length or text_size == 0:
            _text_tokens.append(text[text_size * max_length:])
            _labels.append(label)
    return _text_tokens, np.array(_labels)

In [26]:
X_val

,text
94245,"Licence fees to watch this trash, And pay for ..."
76551,Boogie Nights is one of the best films to come...
96265,I got stuck in traffic (I live in Sicily) on t...
75161,This is another enjoyable and entertaining Hit...
16047,What a let down! This started with an intrigui...
...,...
1846,"The Secret of Kells is one of the most unique,..."
13482,When I tell people that I review movies as a h...
3844,Neil Simon's THE ODD COUPLE set up a model for...
88901,I've read all the Dave Robicheaux novels and c...


In [27]:
len(X_val)

46550

In [25]:
tokenizer_obj = Tokenizer()
total_reviews = np.hstack((X_train, X_val))
tokenizer_obj.fit_on_texts(total_reviews)

vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_val_tokens = tokenizer_obj.texts_to_sequences(X_val)

X_train_tokens, y_train = split_long_texts(X_train_tokens, y_train)
X_val_tokens, y_val = split_long_texts(X_val_tokens, y_val)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_tokens, maxlen=max_length, padding='post')

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 46550 and the array at index 1 has size 2450